# Выбор тарифа
## Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Требуется построить модель с максимально большим значением _accuracy_ (минимум - 0.75). _Accuracy_ необходимо рассчитать на тестовой выборке самостоятельно.

## План выполнения задания

1. Открыть и изучить полученный датасет;
2. Выделить обучающую, валидационную и тестовую выборки;
3. Испытать различные модели для классификации, подбирая оптимальные гиперпараметры; выбрать модель, показывающую наибольшее значение *accuracy*;
4. Проверить качество модели на тестовой выборке;
5. Дополнительное задание: проверить модели на вменяемость.

## Описание данных

Файл ___users_behavior.csv___.    
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Ивестные параметры:
- __сalls__ — количество звонков;
- __minutes__ — суммарная длительность звонков в минутах;
- __messages__ — количество sms-сообщений;
- __mb_used__ — израсходованный интернет-трафик в Мб;
- __is_ultra__ — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Шаг 1. Подготовка данных

Импортируем необходимые для работы модули:

In [89]:
import pandas as pd
from IPython.display import display

from sklearn.model_selection import train_test_split               # разбиение датасета на выборки
from sklearn.metrics import accuracy_score                         # показатель точности
from sklearn.preprocessing import scale                            # стандартизация

from sklearn.dummy import DummyClassifier                          # модели
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

R = 10000                                                          # используемое значение random_state

In [1]:
import imblearn

Откроем полученный датасет и выведем для наглядности несколько записей, а также информацию о таблице:

In [90]:
users_behavior = pd.read_csv('/datasets/users_behavior.csv')
display(users_behavior.head())
users_behavior.info()

calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Таблица содержит 3214 записей, разбитых на 5 столбцов. Предобработка не требуется.

Целевым параметром для нашей модели будет столбец __is_ultra__, в котором отражено, какой тариф используется пользователем. Возможные значения данного столбца - *0* (подключен тариф Smart) и *1* (подключен тариф Ultra), соответственно, перед нами стоит задача бинарной классификации. 

Выделим целевой признак в отдельную переменную ***target***, а остальные признаки сохраним в переменной ***features***:

In [91]:
features = users_behavior.drop(columns = 'is_ultra')
target   = users_behavior['is_ultra']

Для корректной работы некоторых алгоритмов обучения требуется, чтобы признаки были нормированы. Произведем их стандартизацию с помощью функции scale из модуля sklearn.preprocessing:

In [92]:
features = scale(features)

Разобьем получившийся датасет на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. Для этого воспользуемся функцией train_test_split из модуля sklearn.model_selection:

In [93]:
tr_and_val_feats, test_feats, tr_and_val_target, test_target = train_test_split(features, 
                                                                                target,
                                                                                train_size = 0.8,   # 4:1
                                                                                random_state = R)

train_feats, valid_feats, train_target, valid_target = train_test_split(tr_and_val_feats, 
                                                                        tr_and_val_target,
                                                                        train_size = 0.75,          # 3:1
                                                                        random_state = R)

print(f'''Размеры выборок:

Обучающая \t {train_feats.shape[0]}
Валидационная \t {valid_feats.shape[0]}
Тестовая \t {test_feats.shape[0]}''')

Размеры выборок:

Обучающая 	 1928
Валидационная 	 643
Тестовая 	 643


## Шаг 2. Испытание различных моделей обучения

### 1. Критерий оценки

Прежде, чем перейти к рассмотрению различных моделей, нам следует определить критерий вменяемости модели. С помощью классификатора DummyClassifier из состава sklearn.dummy проверим, какой точностью будут обладать некоторые примитивные модели классификации. 

Создадим две модели: первая всегда будет выдавать *0* (наиболее часто встречающийся целевой признак), а вторая - предсказывать результат случайным образом в соответствии с весами классов в датасете - и найдем для них значение метрики **accuracy**:

In [94]:
model_0   = DummyClassifier(strategy = 'most_frequent')
model_str = DummyClassifier(strategy = 'stratified')

model_0.fit(features, target)
print(f'Метрика accuracy для модели 1 (наиболее частое значение):\t\t {model_0.score(features, target):.4f}')

# Для "случайной" модели подсчитаем среднюю точность по итогам нескольких испытаний
accuracies = []
for _ in range(30):
    model_str.fit(features, target)
    accuracies.append(model_str.score(features, target))
print(f'Метрика accuracy для модели 2 (пропорциональный случайный выбор):\t {(sum(accuracies)/30):.4f}')

Метрика accuracy для модели 1 (наиболее частое значение):		 0.6935
Метрика accuracy для модели 2 (пропорциональный случайный выбор):	 0.5727


Таким образом, в дальнейшем нам нужно исходить из того, что точность предсказаний нашей модели должна быть не менее *0.69*.

### 2. Исследование моделей

Для решения нашей задачи рассмотрим следующие алгоритмы классификации:
1. Логистическая регрессия;
2. Наивный байесовский алгоритм;
3. К ближайших соседей;
4. Метод опорных векторов;
5. Решающее дерево;
6. Случайный лес.

Каждую модель мы будем обучать на тренировочном наборе, а затем определять точность (accuracy) ее предсказаний на валидационных данных.
Полученные значения точности будем сохранять в переменную **results**.

In [95]:
results = {}

#### Логистическая регрессия

In [96]:
logreg_model = LogisticRegression(solver = 'liblinear')
logreg_model.fit(train_feats, train_target)
prediction = logreg_model.predict(valid_feats)
results['Логистическая регрессия'] = accuracy_score(valid_target, prediction)

print('Метрика accuracy для модели "Логистическая регрессия": {:.4f}'
      .format(results['Логистическая регрессия']))

Метрика accuracy для модели "Логистическая регрессия": 0.7683


#### Наивный алгоритм Байеса

In [97]:
nb_model = GaussianNB()
nb_model.fit(train_feats, train_target)
prediction = nb_model.predict(valid_feats)
results['Наивный байесовский алгоритм'] = accuracy_score(valid_target, prediction)

print('Метрика accuracy для модели "Наивный байесовский алгоритм": {:.4f}'
      .format(results['Наивный байесовский алгоритм']))

Метрика accuracy для модели "Наивный байесовский алгоритм": 0.7963


#### К-ближайших соседей

In [98]:
knn_model = KNeighborsClassifier()
knn_model.fit(train_feats, train_target)
prediction = knn_model.predict(valid_feats)
results['К соседей'] = accuracy_score(valid_target, prediction)
print('Метрика accuracy для модели "К ближайших соседей": {:.4f}'
      .format(results['К соседей']))

Метрика accuracy для модели "К ближайших соседей": 0.8009


#### Метод опорных векторов

In [99]:
SVM_model = SVC(gamma = 'auto', random_state = R)
SVM_model.fit(train_feats, train_target)
prediction = SVM_model.predict(valid_feats)
results['Опорные векторы'] = accuracy_score(valid_target, prediction)
print('Метрика accuracy для модели "Метод опорных векторов": {:.4f}'
      .format(results['Опорные векторы']))

Метрика accuracy для модели "Метод опорных векторов": 0.8134


#### Решающее дерево

In [100]:
tree_model = DecisionTreeClassifier(random_state = R)
tree_model.fit(train_feats, train_target)
prediction = tree_model.predict(valid_feats)
results['Решающее дерево'] = accuracy_score(valid_target, prediction)
print('Метрика accuracy для модели "Решающее дерево": {:.4f}'
      .format(results['Решающее дерево']))

Метрика accuracy для модели "Решающее дерево": 0.7341


#### Случайный лес

In [101]:
forest_model = RandomForestClassifier(n_estimators = 10, random_state = R)
forest_model.fit(train_feats, train_target)
prediction = forest_model.predict(valid_feats)
results['Случайный лес'] = accuracy_score(valid_target, prediction)
print('Метрика accuracy для модели "Случайный лес": {:.4f}'
      .format(results['Случайный лес']))

Метрика accuracy для модели "Случайный лес": 0.7792


### 3. Подбор гиперпараметров

В итоге нами получены следующие метрики для различных моделей (с параметрами по умолчанию):

In [102]:
results_table = pd.DataFrame(data = results.values(), index = results.keys(), columns = ['accuracy'])
results_table.sort_values(by = 'accuracy', ascending = False)

accuracy
Опорные векторы               0.813375
К соседей                     0.800933
Наивный байесовский алгоритм  0.796267
Случайный лес                 0.779160
Логистическая регрессия       0.768274
Решающее дерево               0.734059

Попробуем улучшить работу модели "Случайный лес". Сперва подберем оптимальные значения количества деревьев (*n_estimators*) и максимальной глубины (*max_depth*):

In [103]:
grid = []
for depth in range (5, 10):
    for estimators in range (20, 101, 10):
        forest_model = RandomForestClassifier(n_estimators = estimators, max_depth = depth, random_state = R)
        forest_model.fit(train_feats, train_target)
        prediction = forest_model.predict(valid_feats)
        grid.append([accuracy_score(valid_target, prediction), estimators, depth])
                       
best = max(grid)
print(f'Максимальное значение accuracy равно {best[0]:.4f},', 
     f'параметры: n_estimators = {best[1]}, max_depth = {best[2]}')

Максимальное значение accuracy равно 0.8196, параметры: n_estimators = 50, max_depth = 9


Как можно видеть, подбор оптимальных гиперпараметров увеличил точность нашей модели с *0.78* до *0.82*, в результате чего она обогнала по данной метрике все остальные рассмотренные классификаторы. 


## Шаг 3. Проверка модели на тестовом наборе

Проверим, как наша модель "Случайный лес", показавшая себя лучше всего на валидационных данных, будет работать на тестовом наборе. Для этого сначала обучим ее с использованием валидационного и тренировочного набора, после чего найдем accuracy для тестовых данных:

In [104]:
forest_model.fit(tr_and_val_feats, tr_and_val_target)
prediction = forest_model.predict(test_feats)
final_score = accuracy_score(test_target, prediction)
print(f'Итоговое значение accuracy - {final_score:.4f}')

Итоговое значение accuracy - 0.8025


Как видно, в итоге мы получили значение accuracy чуть меньше, чем на валидационной выборке, но при этом выше нашей примитивной модели (*0.69*), а также требований задания (*0.75*).

## Вывод

В данной работе перед нами стояла задача построить модель, которая будет выбирать подходящий тариф для клиентов оператора мобильной связи. В нашем распоряжении была информация о поведении пользователей, которые уже перешли на один из двух тарифов.
Для решения этой задачи мы сделали следующее:
1. Выделили целевой признак (тариф - Ultra или Smart) и сохранили его в отдельную переменную;
2. Нормализовали признаки для корректной работы требующих этого алгоритмов (SVM, К ближайших соседей и прочих);
3. Разбили наш датасет на три части: тренировочную, валидационную и тестовую;
4. Выбрали метрику, по которой будем оценивать качество модели (accuracy) и определили ее минимально разумное значение (оно равно 0.69 и получается, если классификатор независимо от всех признаков будет возвращать *0*);
5. Рассмотрели несколько различных алгоритмов классификации и выбрали из них модель "Случайный лес" (Random Forest Classifier). После подбора оптимальных гиперпараметров (n_estimators = 50, max_depth = 9, остальные - по умолчанию) наша модель показала точность на валидационном наборе 0.82;

Качество получившейся модели мы проверили на тестовом наборе, в результате чего итоговое значение accuracy оказалось равным 0.80, что немного ниже, чем на валидационных данных (произошло незначительное переобучение), однако удовлетворяет условиям задания (точность должна быть не менее 0.75).